# Implementación de un sistema de búsqueda genética basado en LangChain y Elasticsearch

## Paso 1: Configuración del Entorno
Instalamos las dependencias necesarias:

In [1]:
pip install beautifulsoup4 eland elasticsearch huggingface-hub langchain tqdm torch requests sentence_transformers googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


## Paso 2: Traducción de la Pregunta
Utilizaremos la librería `googletrans` para traducir la pregunta:

In [2]:
pregunta = "¿Cuál es la causa genética de la fibrosis quística?"

In [3]:
from googletrans import Translator

def translate_to_english(text):
    translator = Translator()
    translation = translator.translate(text, src='es', dest='en')
    return translation.text

question_es = pregunta
question_en = translate_to_english(question_es)
print(question_en)

What is the genetic cause of cystic fibrosis?


## Paso 3
### Búsqueda de Datos en ClinVar y Carga en Elasticsearch

Este código realiza la búsqueda de datos genéticos específicos sobre una enfermedad en la base de datos de ClinVar, extrae la información relevante y la indexa en Elasticsearch para permitir búsquedas semánticas posteriores.

#### Pasos Detallados:

1. **Configuración de Elasticsearch**:
   - **Establecer Conexión**: Configuramos la conexión a Elasticsearch utilizando las credenciales y el endpoint proporcionados.
   - **Verificación de Conexión**: Verificamos que la conexión a Elasticsearch es exitosa usando el método `ping`.
   - **Creación del Índice**: Creamos un índice llamado `genetic_information` en Elasticsearch si no existe ya.

2. **Consulta a la API de ClinVar**:
   - **Buscar Enfermedad**: Realizamos una consulta en la API de ClinVar para buscar registros relacionados con una enfermedad específica (por ejemplo, "Cystic Fibrosis").
   - **Obtener Detalles**: Usamos los identificadores de los registros obtenidos en la búsqueda inicial para hacer otra solicitud a la API y obtener detalles más específicos sobre cada registro.

3. **Procesamiento y Carga de Datos en Elasticsearch**:
   - **Extracción de Información Relevante**: Procesamos la respuesta de la API para extraer datos relevantes como el título del registro, resumen, genes y mutaciones.
   - **Preparación de Datos**: Preparamos los datos en un formato adecuado para la indexación en Elasticsearch.
   - **Carga de Datos**: Usamos la función `helpers.bulk` de la biblioteca `elasticsearch` para cargar los datos extraídos en el índice de Elasticsearch.

In [4]:
enfermedad = "Cystic Fibrosis"

In [5]:
import requests
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url =  f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={disease}&retmode=json"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        ids = result["esearchresult"]["idlist"]
        details_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
        details_response = requests.get(details_url)
        if details_response.status_code == 200:
            return details_response.json()
    return None

# Función para cargar datos en Elasticsearch
def load_data_to_es(data, index_name):
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    helpers.bulk(es, actions)

# Proceso principal para buscar una enfermedad y cargar los datos
disease = enfermedad
data = fetch_clinvar_data(disease)
if data:
    records = []
    for uid in data['result']['uids']:
        record = data['result'][uid]
        records.append({
            "disease": disease,
            "title": record.get("title"),
            "summary": record.get("summary"),
            "gene": record.get("gene"),
            "mutation": record.get("variation")
        })
    load_data_to_es(records, index_name)
    print(f"Datos de {disease} cargados exitosamente en Elasticsearch")
else:
    print(f"No se encontraron datos para {disease}")


Conexión exitosa a Elasticsearch
Datos de Cystic Fibrosis cargados exitosamente en Elasticsearch


### Código para Consultar la API de ClinVar y Obtener Información Completa sobre una Enfermedad
Para simplificar el proceso, creamos una celda que consulte directamente la API de ClinVar para obtener toda la información posible sobre una enfermedad específica y la imprima en formato JSON para su inspección.


### Ejecución del Código

1. **Consulta a la API de ClinVar**:
   - **Función `fetch_clinvar_data(disease)`**: Realiza una consulta en la API de ClinVar para obtener registros relacionados con una enfermedad específica.
     - Utiliza `esearch.fcgi` para obtener los identificadores de los registros.
     - Utiliza `esummary.fcgi` para obtener detalles específicos de cada registro utilizando los identificadores obtenidos.

2. **Impresión de Datos**:
   - Guarda el json para uso posterior e imprime toda la información de la respuesta en formato JSON para inspección.



In [18]:
import requests
import json

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={disease}&retmode=json"
    esearch_response = requests.get(esearch_url)
    if esearch_response.status_code == 200:
        esearch_result = esearch_response.json()
        ids = esearch_result["esearchresult"]["idlist"]
        
        if ids:
            esummary_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
            esummary_response = requests.get(esummary_url)
            if esummary_response.status_code == 200:
                return esummary_response.json()
    return None

# Proceso principal para buscar una enfermedad y mostrar los datos
disease = "Cystic Fibrosis"
data = fetch_clinvar_data(disease)
if data:
    records = []
    for uid in data['result']['uids']:
        record = data['result'][uid]
        
        # Extraer información detallada sobre genes y mutaciones
        gene_info = 'N/A'
        if "genes" in record:
            genes = record["genes"]
            if genes:
                gene_info = genes[0].get('symbol', 'N/A')

        mutation_info = 'N/A'
        if "variation_set" in record:
            variations = record["variation_set"]
            if variations:
                mutation_info = variations[0].get('variation_name', 'N/A')
        
        # Intentar obtener la descripción o proporcionar una predeterminada
        description = record.get("description", "No description available.")
        if description == "N/A":
            description = record.get("summary", "No additional information provided.")

        record_data = {
            "disease": disease,
            "gene": gene_info,
            "mutation": mutation_info,
            "description": description
        }
        
        records.append(record_data)
    
    # Guardar los datos en un archivo JSON
    with open(f"{disease.replace(' ', '_').lower()}_data.json", 'w') as json_file:
        json.dump(records, json_file, indent=2)
    
    print(f"Datos de {disease} guardados exitosamente en {disease.replace(' ', '_').lower()}_data.json")
else:
    print(f"No se encontraron datos para {disease}")

# Ejemplo de lectura del archivo JSON para su posterior uso
with open(f"{disease.replace(' ', '_').lower()}_data.json", 'r') as json_file:
    loaded_data = json.load(json_file)
    for record in loaded_data:
        print(json.dumps(record, indent=2))


Datos de Cystic Fibrosis guardados exitosamente en cystic_fibrosis_data.json
{
  "disease": "Cystic Fibrosis",
  "gene": "WDR19",
  "mutation": "NM_025132.4(WDR19):c.716+2T>C",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "TMEM67",
  "mutation": "NM_153704.6(TMEM67):c.2878G>T (p.Ala960Ser)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "NPHP4",
  "mutation": "NM_015102.5(NPHP4):c.3959T>C (p.Leu1320Pro)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "CFTR",
  "mutation": "NM_000492.4(CFTR):c.2019_2022dup (p.Ala675fs)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "NPHP1",
  "mutation": "NM_001128178.3(NPHP1):c.1255_1258del (p.Ser419fs)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "CFTR",
  "mutation": "NC_000007.13:g.(?_117120078)_(117176728_117180153)dup",
 

## 1a PRUEBA CLON CON MEJORAS
Ajustamos el término de búsqueda para que coincida con los filtros que hemos usado manualmente. Implementaremos el término de búsqueda especificado: 

diabetes [disease/phenotype] AND (("clinsig pathogenic"[Properties] or "clinsig pathogenic low penetrance"[Properties] or "clinsig established risk allele"[Properties]) AND 0[VARLEN]:49[VARLEN]).

**Aquí imprimimos también el código raw que nos devuelve Clnvar, en formato XML**

In [21]:
import requests
import json
import pandas as pd

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    # Término de búsqueda con los filtros especificados
    search_term = f'{disease} [disease/phenotype] AND (("clinsig pathogenic"[Properties] or "clinsig pathogenic low penetrance"[Properties] or "clinsig established risk allele"[Properties]) AND 0[VARLEN]:49[VARLEN])'
    esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={search_term}&retmode=json"
    esearch_response = requests.get(esearch_url)
    print(f"Esearch URL: {esearch_url}")
    print(f"Esearch Response Code: {esearch_response.status_code}")
    print(f"Esearch Response Text: {esearch_response.text}")
    
    if esearch_response.status_code == 200:
        esearch_result = esearch_response.json()
        ids = esearch_result["esearchresult"]["idlist"]
        print(f"Found IDs: {ids}")
        
        if ids:
            esummary_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
            esummary_response = requests.get(esummary_url)
            print(f"Esummay URL: {esummary_url}")
            print(f"Esummay Response Code: {esummary_response.status_code}")
            print(f"Esummay Response Text: {esummary_response.text}")
            
            if esummary_response.status_code == 200:
                return esummary_response.json()
    return None

# Proceso principal para buscar una enfermedad y mostrar los datos
def process_data(disease):
    data = fetch_clinvar_data(disease)
    if data:
        records = []
        for uid in data['result']['uids']:
            record = data['result'][uid]
            
            # Extraer información detallada sobre genes y mutaciones
            gene_info = 'N/A'
            if "genes" in record:
                genes = record["genes"]
                if genes:
                    gene_info = genes[0].get('symbol', 'N/A')

            mutation_info = 'N/A'
            if "variation_set" in record:
                variations = record["variation_set"]
                if variations:
                    mutation_info = variations[0].get('variation_name', 'N/A')
            
            # Intentar obtener la descripción o proporcionar una predeterminada
            description = record.get("description", "No description available.")
            if description == "N/A":
                description = record.get("summary", "No additional information provided.")
            
            # Crear enlace a la información detallada
            link = f"https://www.ncbi.nlm.nih.gov/clinvar/variation/{uid}/"

            record_data = {
                "Gene": gene_info,
                "Mutation": mutation_info,
                "Description": description,
                "Link": link
            }
            
            records.append(record_data)
        
        # Convertir los datos en un DataFrame de pandas
        df = pd.DataFrame(records)
        return df
    else:
        print(f"No se encontraron datos para {disease}")
        return None

# Ejemplo de uso
disease = "diabetes"
df = process_data(disease)
if df is not None:
    df.to_json(f"{disease.replace(' ', '_').lower()}_data.json", orient='records', indent=2)
    print(f"Datos de {disease} guardados exitosamente en {disease.replace(' ', '_').lower()}_data.json")
    print(df)
else:
    print("No se encontraron datos para la enfermedad especificada.")


Esearch URL: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term=diabetes [disease/phenotype] AND (("clinsig pathogenic"[Properties] or "clinsig pathogenic low penetrance"[Properties] or "clinsig established risk allele"[Properties]) AND 0[VARLEN]:49[VARLEN])&retmode=json
Esearch Response Code: 200
Esearch Response Text: {"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"917","retmax":"20","retstart":"0","idlist":["3237529","3236049","3235201","3234111","3234058","3234002","3233996","3233994","3233993","3233992","3233458","3230480","3230478","3068532","3068531","3068530","3068527","3068513","3066243","3066162"],"translationset":[],"translationstack":[{"term":"diabetes[disease/phenotype]","field":"disease/phenotype","count":"6578","explode":"N"},{"term":"\"clinsig pathogenic\"[Properties]","field":"Properties","count":"209630","explode":"N"},{"term":"\"clinsig pathogenic low penetrance\"[Properties]","field":"Properties","count":"16","explode":

## 2a PRUEBA CLON

Limpiamos resultados y arrglamos el pandas de salida

In [22]:
import requests
import json
import pandas as pd

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    # Término de búsqueda con los filtros especificados
    search_term = f'{disease} [disease/phenotype] AND (("clinsig pathogenic"[Properties] or "clinsig pathogenic low penetrance"[Properties] or "clinsig established risk allele"[Properties]) AND 0[VARLEN]:49[VARLEN])'
    esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={search_term}&retmode=json"
    esearch_response = requests.get(esearch_url)
    print(f"Esearch URL: {esearch_url}")
    print(f"Esearch Response Code: {esearch_response.status_code}")
    print(f"Esearch Response Text: {esearch_response.text}")
    
    if esearch_response.status_code == 200:
        esearch_result = esearch_response.json()
        ids = esearch_result["esearchresult"]["idlist"]
        print(f"Found IDs: {ids}")
        
        if ids:
            esummary_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
            esummary_response = requests.get(esummary_url)
            print(f"Esummay URL: {esummary_url}")
            print(f"Esummay Response Code: {esummary_response.status_code}")
            print(f"Esummay Response Text: {esummary_response.text}")
            
            if esummary_response.status_code == 200:
                return esummary_response.json()
    return None

# Proceso principal para buscar una enfermedad y mostrar los datos
def process_data(disease):
    data = fetch_clinvar_data(disease)
    if data:
        records = []
        for uid in data['result']['uids']:
            record = data['result'][uid]
            
            # Extraer información detallada sobre genes y mutaciones
            gene_info = 'N/A'
            if "genes" in record:
                genes = record["genes"]
                if genes:
                    gene_info = genes[0].get('symbol', 'N/A')

            mutation_info = 'N/A'
            if "variation_set" in record:
                variations = record["variation_set"]
                if variations:
                    mutation_info = variations[0].get('variation_name', 'N/A')
            
            # Intentar obtener la descripción o proporcionar una predeterminada
            description = record.get("description", "No description available.")
            if description == "N/A":
                description = record.get("summary", "No additional information provided.")
            
            # Extraer cambios en la proteína
            protein_change = record.get('protein_change', 'N/A')
            
            # Extraer clasificación germinal
            germline_classification = record.get('germline_classification', {}).get('description', 'N/A')

            # Extraer tipo de variante
            variant_type = 'N/A'
            if "variation_set" in record:
                variations = record["variation_set"]
                if variations:
                    variant_type = variations[0].get('variant_type', 'N/A')
            
            # Extraer condición
            condition = 'N/A'
            if "trait_set" in record.get('germline_classification', {}):
                traits = record['germline_classification']['trait_set']
                if traits:
                    condition = traits[0].get('trait_name', 'N/A')
            
            # Crear enlace a la información detallada
            link = f"https://www.ncbi.nlm.nih.gov/clinvar/variation/{uid}/"

            record_data = {
                "Gene": gene_info,
                "Mutation": mutation_info,
                "Protein Change": protein_change,
                "Variant Type": variant_type,
                "Condition": condition,
                "Germline Classification": germline_classification,
                "Description": description,
                "Link": link
            }
            
            records.append(record_data)
        
        # Convertir los datos en un DataFrame de pandas
        df = pd.DataFrame(records)
        return df
    else:
        print(f"No se encontraron datos para {disease}")
        return None

# Ejemplo de uso
disease = "diabetes"
df = process_data(disease)
if df is not None:
    df.to_json(f"{disease.replace(' ', '_').lower()}_data.json", orient='records', indent=2)
    print(f"Datos de {disease} guardados exitosamente en {disease.replace(' ', '_').lower()}_data.json")
    # Mostrar el DataFrame de manera clara
    with pd.option_context('display.max_colwidth', None):
        display(df)
else:
    print("No se encontraron datos para la enfermedad especificada.")


Esearch URL: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term=diabetes [disease/phenotype] AND (("clinsig pathogenic"[Properties] or "clinsig pathogenic low penetrance"[Properties] or "clinsig established risk allele"[Properties]) AND 0[VARLEN]:49[VARLEN])&retmode=json
Esearch Response Code: 200
Esearch Response Text: {"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"917","retmax":"20","retstart":"0","idlist":["3237529","3236049","3235201","3234111","3234058","3234002","3233996","3233994","3233993","3233992","3233458","3230480","3230478","3068532","3068531","3068530","3068527","3068513","3066243","3066162"],"translationset":[],"translationstack":[{"term":"diabetes[disease/phenotype]","field":"disease/phenotype","count":"6578","explode":"N"},{"term":"\"clinsig pathogenic\"[Properties]","field":"Properties","count":"209630","explode":"N"},{"term":"\"clinsig pathogenic low penetrance\"[Properties]","field":"Properties","count":"16","explode":

,Gene,Mutation,Protein Change,Variant Type,Condition,Germline Classification,Description,Link
0,HNF1A,NM_000545.8(HNF1A):c.1324C>T (p.Gln442Ter),Q442*,single nucleotide variant,Maturity-onset diabetes of the young type 3,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3237529/
1,HNF1B,NM_000458.4(HNF1B):c.827_837del (p.Arg276fs),"R250fs, R276fs",Deletion,Renal cysts and diabetes syndrome,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3236049/
2,HNF1B,NM_000458.4(HNF1B):c.92_99dup (p.Leu34fs),L34fs,Duplication,Renal cysts and diabetes syndrome,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3235201/
3,HNF1B,NM_000458.4(HNF1B):c.1397_1404del (p.Leu466fs),"L440fs, L466fs",Deletion,Renal cysts and diabetes syndrome,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3234111/
4,AVPR2,NM_000054.7(AVPR2):c.575G>C (p.Cys192Ser),C192S,single nucleotide variant,"Diabetes insipidus, nephrogenic, X-linked",Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3234058/
5,GCK,NM_000162.5(GCK):c.605T>G (p.Met202Arg),"M201R, M202R, M203R",single nucleotide variant,Monogenic diabetes,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3234002/
6,GCK,NM_000162.5(GCK):c.1228G>T (p.Gly410Cys),"G30C, G409C, G410C, G411C, G73C, G88C",single nucleotide variant,Monogenic diabetes,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3233996/
7,GCK,NM_000162.5(GCK):c.1234G>A (p.Val412Met),"V32M, V411M, V412M, V413M, V75M, V90M",single nucleotide variant,Monogenic diabetes,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3233994/
8,GCK,NM_000162.5(GCK):c.454T>A (p.Phe152Ile),"F151I, F152I, F153I",single nucleotide variant,Monogenic diabetes,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3233993/
9,GCK,NM_000162.5(GCK):c.1235T>A (p.Val412Glu),"V32E, V411E, V412E, V413E, V75E, V90E",single nucleotide variant,Monogenic diabetes,Pathogenic,No description available.,https://www.ncbi.nlm.nih.gov/clinvar/variation/3233992/


## MEJORAMOS LA TABLA DE SALIDA

### Comparativa entre el uso de la API de ClinVar y Elasticsearch

En nuestro proyecto, tenemos dos enfoques para obtener y presentar datos genéticos relacionados con enfermedades: el uso directo de la API de ClinVar y el uso de Elasticsearch para manejar y buscar grandes volúmenes de datos. A continuación, describimos las diferencias entre ambos enfoques y sus implicaciones para nuestro trabajo.

#### Uso de la API de ClinVar

**Ventajas:**

1. **Simplicidad**: El código es más simple y directo, sin necesidad de configurar y mantener un clúster de Elasticsearch.
2. **Menos Dependencias**: Al utilizar solo la API de ClinVar, reducimos la cantidad de dependencias y posibles puntos de falla.
3. **Facilidad de Implementación**: El código es más fácil de implementar y mantener, especialmente si no tenemos experiencia previa con Elasticsearch.
4. **Acceso Directo a Datos Actualizados**: Obtenemos los datos directamente de ClinVar, asegurando que siempre trabajamos con la información más actualizada disponible.

**Desventajas:**

1. **Limitaciones en Búsquedas Complejas**: La API de ClinVar puede tener limitaciones en la realización de búsquedas complejas comparado con las capacidades de Elasticsearch.
2. **Rendimiento en Búsqueda**: Para consultas muy frecuentes o que manejan grandes volúmenes de datos, el rendimiento puede no ser tan eficiente como con Elasticsearch.

#### Uso de Elasticsearch

**Ventajas:**

1. **Rendimiento en Búsqueda**: Elasticsearch es muy eficiente para realizar búsquedas complejas y rápidas sobre grandes volúmenes de datos.
2. **Escalabilidad**: Elasticsearch puede manejar fácilmente la escalabilidad y la gestión de grandes conjuntos de datos.
3. **Flexibilidad en las Consultas**: Ofrece una gran flexibilidad en las consultas, lo que puede ser útil si necesitamos realizar búsquedas más complejas o específicas en el futuro.
4. **Capacidades de Análisis Avanzado**: Permite realizar análisis y agregaciones avanzadas que pueden proporcionar insights adicionales sobre los datos.

**Desventajas:**

1. **Complejidad**: Configurar y mantener un clúster de Elasticsearch añade complejidad a nuestro proyecto.
2. **Dependencias Adicionales**: Necesitamos gestionar credenciales y asegurar la conectividad a Elasticsearch.
3. **Mantenimiento**: Requiere mantenimiento continuo del clúster y de los índices de datos.
4. **Duración de la cuentra free**: Desde que la abres y creas el ´cluster´ elastic te da solo 21 días gratis.

### Conclusión

Si nuestro objetivo principal es obtener datos específicos de ClinVar y presentarlos de manera clara, podemos simplificar nuestro flujo de trabajo utilizando solo la API de ClinVar. Sin embargo, si prevemos la necesidad de realizar búsquedas complejas y escalables en el futuro, mantener Elasticsearch podría ser beneficioso.

## A continuación, presentamos el código para integrar y buscar datos utilizando Elasticsearch.


### Utilización de la API de ClinVar

**NOTA: Ahora usa la info del json grabado en la celda anterior**

La API de ClinVar permite acceder a datos sobre variantes genéticas y su relación con enfermedades. Usaremos las funciones `esearch` y `esummary` de la API de Entrez para obtener estos datos.

### Explicación del Código

1. **Configuración de Elasticsearch**:
   - Configuramos la conexión a Elasticsearch utilizando las credenciales y el endpoint proporcionados.
   - Verificamos la conexión y crea un índice si no existe.

2. **Consulta a la API de ClinVar**:
   - **Función `fetch_clinvar_data(disease)`**: Realiza una consulta en la API de ClinVar para obtener registros relacionados con una enfermedad específica.
     - Utiliza `esearch.fcgi` para obtener los identificadores de los registros.
     - Utiliza `esummary.fcgi` para obtener detalles específicos de cada registro utilizando los identificadores obtenidos.

3. **Procesamiento de Datos**:
   - Extrae la información relevante de cada registro, incluyendo el título, resumen, genes y mutaciones.
   - Asegura que al menos uno de los campos relevantes no sea `None` o `N/A` antes de agregar el registro a la lista de datos para cargar.

4. **Carga de Datos en Elasticsearch**:
   - Prepara y carga los datos en Elasticsearch utilizando la función `helpers.bulk`.

5. **Búsqueda y Visualización**:
   - **Función `search_data_in_es(index_name, query)`**: Realiza una búsqueda en el índice `genetic_information` para obtener documentos relacionados con "Cystic Fibrosis".
   - Imprime los resultados de manera legible, manejando los casos en los que algunos campos puedan ser `None`.

### Referencias:
- [ClinVar API Documentation](https://www.ncbi.nlm.nih.gov/clinvar/docs/maintenance_use/#api)
- [Elasticsearch Python Client Documentation](https://elasticsearch-py.readthedocs.io/en/latest/)

## Paso 4: Búsqueda Semántica con LangChain
Configuramos LangChain para realizar la búsqueda semántica en Elasticsearch:

In [25]:
import requests
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url = f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para cargar datos en Elasticsearch desde un archivo JSON
def load_data_from_json(json_file_path, index_name):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    try:
        helpers.bulk(es, actions)
        print(f"Datos cargados exitosamente en Elasticsearch desde {json_file_path}")
    except helpers.BulkIndexError as e:
        print("Bulk indexing error:", e)
        for error in e.errors:
            print(json.dumps(error, indent=2))

# Proceso principal para cargar datos desde el archivo JSON
json_file_path = "cystic_fibrosis_data.json"  # Asegúrarse de que el archivo está en el directorio correcto
load_data_from_json(json_file_path, index_name)

# Función para buscar datos en Elasticsearch
def search_data_in_es(index_name, query):
    search_query = {
        "query": {
            "match": {
                "disease": query
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    return response['hits']['hits']

# Buscar datos de "Cystic Fibrosis"
results = search_data_in_es(index_name, "Cystic Fibrosis")

# Imprimir resultados
for result in results:
    data = result['_source']
    disease = data.get('disease', 'N/A')
    gene = data.get('gene', 'N/A')
    mutation = data.get('mutation', 'N/A')
    description = data.get('description', 'N/A')
    
    print(f"Disease: {disease}")
    print(f"Gene: {gene}")
    print(f"Mutation: {mutation}")
    print(f"Description: {description}")
    print("\n" + "-"*50 + "\n")


Conexión exitosa a Elasticsearch
Datos cargados exitosamente en Elasticsearch desde cystic_fibrosis_data.json
Disease: Cystic Fibrosis
Gene: WDR19
Mutation: NM_025132.4(WDR19):c.716+2T>C
Description: N/A

--------------------------------------------------

Disease: Cystic Fibrosis
Gene: TMEM67
Mutation: NM_153704.6(TMEM67):c.2878G>T (p.Ala960Ser)
Description: N/A

--------------------------------------------------

Disease: Cystic Fibrosis
Gene: NPHP4
Mutation: NM_015102.5(NPHP4):c.3959T>C (p.Leu1320Pro)
Description: N/A

--------------------------------------------------

Disease: Cystic Fibrosis
Gene: CFTR
Mutation: NM_000492.4(CFTR):c.2019_2022dup (p.Ala675fs)
Description: N/A

--------------------------------------------------

Disease: Cystic Fibrosis
Gene: NPHP1
Mutation: NM_001128178.3(NPHP1):c.1255_1258del (p.Ser419fs)
Description: N/A

--------------------------------------------------

Disease: Cystic Fibrosis
Gene: CFTR
Mutation: NC_000007.13:g.(?_117120078)_(117176728_11718

## Paso 5: Prueba cutre de generación de la respuesta sin LLM
Generamos una respuesta combinando la información obtenida con la pregunta original:

In [9]:
import requests
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url = f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para cargar datos en Elasticsearch desde un archivo JSON
def load_data_from_json(json_file_path, index_name):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    try:
        helpers.bulk(es, actions)
        print(f"Datos cargados exitosamente en Elasticsearch desde {json_file_path}")
    except helpers.BulkIndexError as e:
        print("Bulk indexing error:", e)
        for error in e.errors:
            print(json.dumps(error, indent=2))

# Proceso principal para cargar datos desde el archivo JSON
json_file_path = "cystic_fibrosis_data.json"  # Asegúrate de que el archivo está en el directorio correcto
load_data_from_json(json_file_path, index_name)

# Función para buscar datos en Elasticsearch
def search_data_in_es(index_name, query):
    search_query = {
        "query": {
            "match": {
                "disease": query
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    return response['hits']['hits']

# Función para generar una respuesta a partir de los documentos
def generate_response(question, documents):
    relevant_info = " ".join([f"Gene: {doc['_source'].get('gene', 'N/A')}, Mutation: {doc['_source'].get('mutation', 'N/A')}, Description: {doc['_source'].get('description', 'No description available.')}" for doc in documents])
    response = f"Para la pregunta '{question}', la información genética relevante es: {relevant_info}"
    return response

# Buscar datos de "Cystic Fibrosis"
index_name = "genetic_information"
query = "Cystic Fibrosis"
documents = search_data_in_es(index_name, query)

# Generar una respuesta basada en una pregunta
question_es = "¿Cuál es la información genética relevante para la fibrosis quística?"
response = generate_response(question_es, documents)
print(response)


Conexión exitosa a Elasticsearch
Datos cargados exitosamente en Elasticsearch desde cystic_fibrosis_data.json
Para la pregunta '¿Cuál es la información genética relevante para la fibrosis quística?', la información genética relevante es: Gene: WDR19, Mutation: NM_025132.4(WDR19):c.716+2T>C, Description: No description available. Gene: TMEM67, Mutation: NM_153704.6(TMEM67):c.2878G>T (p.Ala960Ser), Description: No description available. Gene: NPHP4, Mutation: NM_015102.5(NPHP4):c.3959T>C (p.Leu1320Pro), Description: No description available. Gene: CFTR, Mutation: NM_000492.4(CFTR):c.2019_2022dup (p.Ala675fs), Description: No description available. Gene: NPHP1, Mutation: NM_001128178.3(NPHP1):c.1255_1258del (p.Ser419fs), Description: No description available. Gene: CFTR, Mutation: NC_000007.13:g.(?_117120078)_(117176728_117180153)dup, Description: No description available. Gene: CFTR, Mutation: NM_000492.4(CFTR):c.202A>C (p.Lys68Gln), Description: No description available. Gene: CFTR, Mut

# EJEMPLO FINAL CON TODO EL FLUJO DE TRABAJO CON ELASTICSEARCH
Este código graba el json y lo lee, todo en el mismo código

## Uso de Elasticsearch para Consulta y Búsqueda de Información Genética

En esta celda, hemos integrado todo el flujo de trabajo necesario para obtener información genética de la API de ClinVar, procesar estos datos, guardarlos en un archivo JSON, cargar este archivo en Elasticsearch y finalmente realizar búsquedas en Elasticsearch. A continuación se detalla cada paso del proceso:

# Ejemplo de uso
disease = "Cystic Fibrosis"  # Cambiar a la enfermedad deseada

1. **Conexión a Elasticsearch**:
   - Establecemos la conexión a Elasticsearch utilizando las credenciales proporcionadas.
   - Verificamos la conexión para asegurarnos de que Elasticsearch está accesible.

2. **Crear Índice en Elasticsearch**:
   - Comprobamos si el índice `genetic_information` existe.
   - Si no existe, lo creamos para almacenar la información genética.

3. **Obtener Datos de ClinVar**:
   - Consultamos la API de ClinVar para obtener datos relacionados con la enfermedad especificada.
   - Procesamos los datos obtenidos para extraer información relevante sobre genes, mutaciones y descripciones.

4. **Guardar Datos en JSON**:
   - Guardamos los datos procesados en un archivo JSON local. Esto nos permite tener una copia de los datos para futuras referencias y para cargar en Elasticsearch.

5. **Cargar Datos en Elasticsearch**:
   - Utilizamos la función `helpers.bulk` de Elasticsearch para cargar los datos desde el archivo JSON en el índice `genetic_information`.
   - Gestionamos posibles errores durante la carga masiva de datos.

6. **Buscar Datos en Elasticsearch**:
   - Realizamos una búsqueda en Elasticsearch para obtener los registros relacionados con la enfermedad especificada.
   - Mostramos los resultados obtenidos de Elasticsearch, incluyendo la enfermedad, el gen, la mutación y la descripción.


import requests
import json
from elasticsearch import Elasticsearch, helpers

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url = f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch si no existe
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={disease}&retmode=json"
    esearch_response = requests.get(esearch_url)
    if esearch_response.status_code == 200:
        esearch_result = esearch_response.json()
        ids = esearch_result["esearchresult"]["idlist"]
        
        if ids:
            esummary_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
            esummary_response = requests.get(esummary_url)
            if esummary_response.status_code == 200:
                return esummary_response.json()
    return None

# Función para procesar datos obtenidos de ClinVar
def process_clinvar_data(data, disease):
    records = []
    for uid in data['result']['uids']:
        record = data['result'][uid]
        
        gene_info = 'N/A'
        if "genes" in record:
            genes = record["genes"]
            if genes:
                gene_info = genes[0].get('symbol', 'N/A')

        mutation_info = 'N/A'
        if "variation_set" in record:
            variations = record["variation_set"]
            if variations:
                mutation_info = variations[0].get('variation_name', 'N/A')
        
        description = record.get("description", "No description available.")
        if description == "N/A":
            description = record.get("summary", "No additional information provided.")

        record_data = {
            "disease": disease,
            "gene": gene_info,
            "mutation": mutation_info,
            "description": description
        }
        
        records.append(record_data)
    return records

# Función para guardar datos en un archivo JSON
def save_data_to_json(data, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=2)
    print(f"Datos guardados exitosamente en {file_path}")

# Función para cargar datos en Elasticsearch y manejar errores
def load_data_to_es(data, index_name):
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    try:
        helpers.bulk(es, actions)
        print("Datos cargados exitosamente en Elasticsearch")
    except helpers.BulkIndexError as e:
        print("Bulk indexing error:", e)
        for error in e.errors:
            print(json.dumps(error, indent=2))

# Función para buscar datos en Elasticsearch
def search_data_in_es(index_name, query):
    search_query = {
        "query": {
            "match": {
                "disease": query
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    return response['hits']['hits']

# Proceso principal
def main(disease):
    # Paso 1: Obtener datos de ClinVar
    data = fetch_clinvar_data(disease)
    if data:
        # Paso 2: Procesar datos obtenidos
        processed_data = process_clinvar_data(data, disease)
        
        # Paso 3: Guardar datos en archivo JSON
        json_file_path = f"{disease.replace(' ', '_').lower()}_data.json"
        save_data_to_json(processed_data, json_file_path)
        
        # Paso 4: Cargar datos en Elasticsearch
        load_data_to_es(processed_data, index_name)
        
        # Paso 5: Buscar y mostrar resultados desde Elasticsearch
        results = search_data_in_es(index_name, disease)
        for result in results:
            data = result['_source']
            print(f"Disease: {data.get('disease', 'N/A')}")
            print(f"Gene: {data.get('gene', 'N/A')}")
            print(f"Mutation: {data.get('mutation', 'N/A')}")
            print(f"Description: {data.get('description', 'N/A')}")
            print("\n" + "-"*50 + "\n")
    else:
        print(f"No se encontraron datos para {disease}")

# Ejemplo de uso
disease = "Cystic Fibrosis"  # Cambiar a la enfermedad deseada
main(disease)
